# **ControlAgent Demo 2**

Welcome to this demonstration notebook, where we showcase the controller design process using our ControlAgent framework. This example focuses on a **second-order unstable system**.

### Workflow Overview

**Extract User Requirements**: We begin by extracting user requirements from ControlEval. This initial step involves gathering the dynamical system models and the corresponding performance requirements.

**Central Agent Analysis**: The central agent processes these requirements to summarize the task, perform a high-level analysis, and assign the design task to a task-specific agent.

**Task-Specific Agent**: A task-specific agent is assigned. This agent is responsible for carrying out iterative design adjustments to meet the predefined criteria effectively.

In [1]:
import control as ctrl
import json
import os
from gpt4 import GPT4
from DesignMemory import Design_Memory

In [2]:
gpt4 = GPT4()

In [3]:
task_id = 0
file_path = './ControlEval/second_order_unstable_data.json'

# Check if the file exists
if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
else:
    print("File does not exist")

# Extract the user request
system = data[task_id]
num = [system['num'][0]]
den = system['den']
try:
    tau = system['tau']
    system_w_delay = True
except:
    system_w_delay = False
plant = ctrl.TransferFunction(num, den)

# Extract performance requirements
phase_margin_min = system['phase_margin_min']
settling_time_min = system['settling_time_min']
settling_time_max = system['settling_time_max']
steadystate_error_max = system['steadystate_error_max']
scenario = system['scenario']

# Encode Feedback
thresholds = {
    'phase_margin': {'min': phase_margin_min, 'message': f'Phase margin should be at least {phase_margin_min} degrees.'},
    'settling_time_min': {'min': settling_time_min, 'message': f'Settling time should be at least {settling_time_min} sec.'},
    'settling_time_max': {'max': settling_time_max, 'message': f'Settling time should be at most {settling_time_max} sec.'},
    'steadystate_error': {'max': steadystate_error_max, 'message': f'Steady state error should be at most {steadystate_error_max}.'}
}

requirements_summary = "\nDesign the controller to meet the following specifications: \n"
requirements_summary += f"Phase margin greater or equal {phase_margin_min} degrees, \n"
requirements_summary += f"Settling time greater or equal {settling_time_min} sec, \n"
requirements_summary += f"Settling time should also be less or equal to {settling_time_max} sec, \n"
requirements_summary += f"Steady state error less or equal {steadystate_error_max}. \n"

if system_w_delay:
    user_request = "\nPlease design the controller for the following system: " +  str(plant) + f" with time delay {tau} sec" + requirements_summary
else:
    user_request = "\nPlease design the controller for the following system: " +  str(plant) + requirements_summary

## Central Agent

In [4]:
from instruction import central_agent_prompt, response_instruct
# Construct input prompt for central agent
central_prompt = central_agent_prompt + user_request + response_instruct
print(f"Input prompt for central agent is:\n {central_prompt}")

Input prompt for central agent is:
 

You are an expert control engineer tasked with analyzing the provided control task and assigning it to the most suitable task-specific agent, each specializing in designing controllers for specific system types.

First, analyze the dynamic system to identify its type, such as a first-order stable system, second-order unstable system, first-order with time delay, higher-order system, etc. Based on this analysis, assign the task to the corresponding task-specific agent that specializes in the identified system type.

Here are the available task-specific agents:
- **Agent 1**: First-order stable system
- **Agent 2**: First-order unstable system
- **Agent 3**: Second-order stable system
- **Agent 4**: Second-order unstable system
- **Agent 5**: First-order system with time delay
- **Agent 6**: Higher-order system

Ensure the selected agent can effectively tailor the control design process.


Please design the controller for the following system: <Trans

In [5]:
# Obtain the response of central agent
central_response = gpt4.complete(central_prompt)

print(f"Central agent response is:\n{central_response}")

# Task assignment based on the response of central agent
parsed_response = json.loads(central_response)
agent_number = int(parsed_response.get("Agent Number"))
task_requirement = parsed_response.get("Task Requirement")
# Define the mapping of agent numbers to their descriptions
sub_agents = {
    1: "first_ord_stable_Design",
    2: "first_ord_unstable_Design",
    3: "second_ord_stable_Design",
    4: "second_ord_unstable_Design",
    5: "first_ord_w_delay_Design",
    6: "higher_ord_Design"
}
# Check if the agent number is in the dictionary and act accordingly
if agent_number in sub_agents:
    sub_agent = sub_agents[agent_number]
    print(f"Activating sub-agent: {agent_number} for {sub_agent}")
else:
    print("No suitable sub-agent found for this task.")

Central agent response is:
{
    "Task Requirement": "Design a controller for a system with the transfer function 10.64 / (s^2 - 1.931s + 3.237) to meet the following specifications: phase margin >= 52.824 degrees, settling time between 0.195 and 5.651 seconds, and steady state error <= 0.0001.",
    "Task Analysis": "The given transfer function is a second-order system with a characteristic equation of s^2 - 1.931s + 3.237. The system is potentially unstable as the denominator's roots need to be checked for stability. Since the system is second-order and potentially unstable, it is best suited for Agent 4, who specializes in second-order unstable systems.",
    "Agent Number": "4"
}
Activating sub-agent: 4 for second_ord_unstable_Design


## Task-Specific Agent for Second-order Unstable Systems


In [6]:
from instruction import overall_instruction_RHP_Pole_PID, response_format_PID
from util import loop_shaping_pid, feedback_prompt, check_stability_pid
# Initialize the design memory
design_memory = Design_Memory()
# Maximum iteration number
max_attempts = 20
# Initilize the iteration number
num_attempt = 1
# Implement the task handling process
print(f"Handling controller design for system {system['id']} scenario {scenario}.\n")

# Construct the design prompt
prompt = overall_instruction_RHP_Pole_PID
new_problem = "Now consider the following design task: " + task_requirement

problem_statement = prompt + new_problem + response_format_PID

while num_attempt <= max_attempts:
    # Call GPT4 to complete the prompt
    print(f"Input prompt for iteration {num_attempt} is: {problem_statement}")
    print('-' * 114)
    response = gpt4.complete(problem_statement)
    print(f"Response for iteration {num_attempt} is:\n {response}")


    data = json.loads(response)

    # parameters for plant transfer function
    num = [system['num'][0]]
    den = system['den']

    # Extract the list of parameters
    parameters = data['parameter']
    omega_L = parameters[0]
    beta_b = parameters[1]
    beta_l = parameters[2]
    is_stable = check_stability_pid(omega_L, beta_b, beta_l, num, den)
    if is_stable:
        _, phase_margin, _, settlingtime, _, sse = loop_shaping_pid(omega_L, beta_b, beta_l, num, den)
        design_memory.add_design(
            parameters={'omega_L': omega_L, 'beta_b': beta_b, 'beta_l': beta_l},
            performance={'phase_margin': phase_margin, 'settling_time_min': settlingtime, 'settling_time_max': settlingtime,'steadystate_error': sse}
        )
        design = design_memory.get_latest_design()
        is_succ = True  # Assume success unless a metric fails
        for metric, specs in thresholds.items():
            value = design['performance'].get(metric)
            if value is not None:
                if 'min' in specs and value < specs['min']:
                    is_succ = False  # Set success to False if any metric fails
                elif 'max' in specs and value > specs['max']:
                    is_succ = False  # Set success to False if any metric fails
        if is_succ:
            print("The current design satisfies the requirement!")
            print(f"Achieved phase Margin is {phase_margin}")
            print(f"Achieved settling Time is {settlingtime}")
            print(f"The steady-state error is {sse}")
            print('=' * 114)

            # Save success information and final design to the log
            final_result = {
                "is_succ": True,
                "parameters": design['parameters'],
                "performance": design['performance'],
                "conversation_rounds": num_attempt
            }

            break
        else:
          print("The current design does not satisfy the requirement.")
          print('=' * 114)

        feedback = feedback_prompt(design_memory, thresholds)
        problem_statement = prompt + new_problem + "\n" + feedback + response_format_PID
    else:
        # Save unstable design information to the log
        feedback = feedback_prompt(design_memory, thresholds)
        problem_statement = prompt + new_problem + "\n" + feedback + response_format_PID
    num_attempt += 1

if num_attempt == max_attempts + 1:
    design = design_memory.get_latest_design()
    # Save failure information and final design to the log
    final_result = {
        "is_succ": False,
        "parameters": design['parameters'],
        "performance": design['performance'],
        "conversation_rounds": num_attempt-1
    }

Handling controller design for system 0 scenario other.

Input prompt for iteration 1 is: 

You are a control engineer expert, and your goal is to design a controller K(s) for a system with transfer function G(s) using loop shaping method.
The system under consideration has unstable pole(s), making it fundamentally more difficult to control.
The loop transfer function is L(s) = G(s)K(s) and here are the basic loop shaping steps for a plant with unstable pole(s):
[Step1] Find the poles of the given plant G(s) and identify the fastest unstable pole, which is the unstable pole with the real part having the largest absolute value. Let's denote this pole as p.
[Step2] Choose a proper loop bandwidth omega_L greater than (5*p) for the given plant G(s). 
Note: Systems with unstable pole(s) puts a hard limit on the achievable bandwidth, you should never choose a loop bandwidth less than (5*p).
Within the valid range, increasing omega_L will make the response faster, therefore smaller settling t